In [1]:
import warnings
warnings.filterwarnings('ignore')


In [2]:
pip install pyspark


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install numpy


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
# importing required libraries
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
import pyspark.sql.types as tp
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, IndexToString
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, CountVectorizer, IDF
from pyspark.ml.classification import LogisticRegression, NaiveBayes
from pyspark.sql import Row

In [5]:
# importing required libraries
from pyspark import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.streaming import StreamingContext
import pyspark.sql.types as tp
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, IndexToString
from pyspark.ml.feature import StopWordsRemover, Word2Vec, RegexTokenizer, CountVectorizer, IDF
from pyspark.ml.classification import LogisticRegression, NaiveBayes
from pyspark.sql import Row

In [6]:
# initializing spark session
sc = SparkContext(appName="PySparkShell")
# spark = SparkSession(sc)
    
# from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("MyApp") \
    .config("spark.driver.memory", "8g") \
    .config("spark.executor.memory", "8g") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/02 21:32:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
# define the schema
my_schema = tp.StructType([
  tp.StructField(name= 'label',       dataType= tp.IntegerType(),  nullable= True),
  tp.StructField(name= 'id',          dataType= tp.IntegerType(),  nullable= True),
  tp.StructField(name= 'date',          dataType= tp.StringType(),  nullable= True),
  tp.StructField(name= 'NO_QUERY',       dataType= tp.StringType(),  nullable= True),
  tp.StructField(name= 'username',       dataType= tp.StringType(),  nullable= True),
  tp.StructField(name= 'tweet',       dataType= tp.StringType(),   nullable= True)
])
    
 

In [8]:
 
# read the dataset  
my_data = spark.read.csv('training.1600000.csv',
                         schema=my_schema,
                         header=True)



In [9]:
my_data=my_data.drop("id","date","NO_QUERY","username")

In [10]:
my_data = my_data.limit(5000)

In [11]:
# my_data = my_data.sample(n=5000, random_state=42)

In [12]:
# view the data
my_data.show(10)

# print the schema of the file
my_data.printSchema()

23/05/02 21:32:10 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 0, @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
 Schema: label, tweet
Expected: label but found: 0
CSV file: file:///Users/vijay_vbr/Downloads/Real_Time_Data_Analysis-main/training.1600000.csv
+-----+--------------------+
|label|               tweet|
+-----+--------------------+
|    0|is upset that he ...|
|    0|@Kenichan I dived...|
|    0|my whole body fee...|
|    0|@nationwideclass ...|
|    0|@Kwesidei not the...|
|    0|         Need a hug |
|    0|@LOLTrish hey  lo...|
|    0|@Tatiana_K nope t...|
|    0|@twittera que me ...|
|    0|spring break in p...|
+-----+--------------------+
only showing top 10 rows

root
 |-- label: integer (nullable = true)
 |-- tweet: string (nullable = true)



In [13]:
# define stage 1: tokenize the tweet text    
stage_1 = RegexTokenizer(inputCol= 'tweet' , outputCol= 'tokens', pattern= '\\W')
# define stage 2: remove the stop words
stage_2 = StopWordsRemover(inputCol= 'tokens', outputCol= 'filtered_words')
# define stage 3: create a word vector of the size 100
# bag of words count
stage_3 = CountVectorizer(inputCol="filtered_words", outputCol="cf", vocabSize=20000, minDF=5)
#hashtf = HashingTF(numFeatures=2 ** 16, inputCol="wordsWithStopwordsfree", outputCol="tf")
stage_4 = IDF(inputCol="cf", outputCol="vector", minDocFreq=5)
stage_3 = Word2Vec(inputCol= 'filtered_words', outputCol= 'vector', vectorSize= 100)
# define stage 4: Logistic Regression Model
model = LogisticRegression(featuresCol= 'vector', labelCol= 'label')

In [14]:

# setup the pipeline
pipeline = Pipeline(stages= [stage_1, stage_2, stage_3, model])

# fit the pipeline model with the training data
pipelineFit = pipeline.fit(my_data)

23/05/02 21:32:11 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
 Schema: tweet
Expected: tweet but found: @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D
CSV file: file:///Users/vijay_vbr/Downloads/Real_Time_Data_Analysis-main/training.1600000.csv
23/05/02 21:32:12 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/05/02 21:32:12 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
23/05/02 21:32:12 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
23/05/02 21:32:12 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 0, @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got

In [16]:
pipelineFit.save("preprocessingAndLR")

In [17]:
# read the dataset  
bbc_data = spark.read.csv('BBCNewsTrain.csv',
                         header=True)



In [18]:
# pipelineFit.save("preprocessingAndLR")

In [19]:
bbc_data.show(10)

+---------+--------------------+-------------+
|ArticleId|                Text|     Category|
+---------+--------------------+-------------+
|     1833|worldcom ex-boss ...|     business|
|      154|german business c...|     business|
|     1101|bbc poll indicate...|     business|
|     1976|lifestyle  govern...|         tech|
|      917|enron bosses in $...|     business|
|     1582|howard  truanted ...|     politics|
|      651|wales silent on g...|        sport|
|     1797|french honour for...|entertainment|
|     2034|car giant hit by ...|     business|
|     1866|fockers fuel fest...|entertainment|
+---------+--------------------+-------------+
only showing top 10 rows



In [20]:
bbc_data=bbc_data.drop('ArticleId')
bbc_data=bbc_data.withColumnRenamed("Text","tweet")

In [21]:
from pyspark.sql.functions import when
bbc_data = bbc_data.withColumn("Category", when(bbc_data.Category == "tech",0) \
      .when(bbc_data.Category == "business",1).when(bbc_data.Category == "politics",2).when(bbc_data.Category == "sport",3).when(bbc_data.Category == "entertainment",4).otherwise(0))

#indexers = [StringIndexer(inputCol="Category", outputCol="Category_index")]


#pipeline = Pipeline(stages=indexers)
#bbc_data = pipeline.fit(bbc_data).transform(bbc_data)


In [22]:
bbc_data.show(10)

+--------------------+--------+
|               tweet|Category|
+--------------------+--------+
|worldcom ex-boss ...|       1|
|german business c...|       1|
|bbc poll indicate...|       1|
|lifestyle  govern...|       0|
|enron bosses in $...|       1|
|howard  truanted ...|       2|
|wales silent on g...|       3|
|french honour for...|       4|
|car giant hit by ...|       1|
|fockers fuel fest...|       4|
+--------------------+--------+
only showing top 10 rows



In [23]:



# define stage 1: tokenize the tweet text    
stage_1 = RegexTokenizer(inputCol= 'tweet' , outputCol= 'tokens', pattern= '\\W')
# define stage 2: remove the stop words
stage_2 = StopWordsRemover(inputCol= 'tokens', outputCol= 'filtered_words')
# define stage 3: create a word vector of the size 100
#stage_3 = Word2Vec(inputCol= 'filtered_words', outputCol= 'vector')

# bag of words count
stage_3 = CountVectorizer(inputCol="filtered_words", outputCol="cf", vocabSize=20000, minDF=5)
#hashtf = HashingTF(numFeatures=2 ** 16, inputCol="wordsWithStopwordsfree", outputCol="tf")
stage_4 = IDF(inputCol="cf", outputCol="vector", minDocFreq=5)
# define stage 4: Logistic Regression Model
model = NaiveBayes(featuresCol= 'vector', labelCol= 'Category',smoothing=1.0, modelType='multinomial')




In [24]:
# setup the pipeline
pipelineBBC = Pipeline(stages= [stage_1, stage_2, stage_3,stage_4,model])



In [25]:
trainDF,testDF=bbc_data.randomSplit([0.75, 0.25], seed=2000)
# fit the pipeline model with the training data
BBCpipelineFit = pipelineBBC.fit(trainDF)

In [26]:
test_df = BBCpipelineFit.transform(testDF)

In [27]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


test_df=test_df.withColumnRenamed("Category","label")
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction")
evaluator.evaluate(test_df)

0.9699242736504335

In [28]:
BBCpipelineFit.save("preprocessingAndCategory")

In [29]:
bbc_data.show(10)

+--------------------+--------+
|               tweet|Category|
+--------------------+--------+
|worldcom ex-boss ...|       1|
|german business c...|       1|
|bbc poll indicate...|       1|
|lifestyle  govern...|       0|
|enron bosses in $...|       1|
|howard  truanted ...|       2|
|wales silent on g...|       3|
|french honour for...|       4|
|car giant hit by ...|       1|
|fockers fuel fest...|       4|
+--------------------+--------+
only showing top 10 rows



In [30]:
#sc.stop()